In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Liver_cirrhosis/GSE182060'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Prognostic liver signature profiles in biopsy tissues from non-alcoholic fatty liver disease patients followed for fibrosis progression"
!Series_summary	"Background/Aims: There is a major unmet need to assess prognostic impact of anti-fibrotics in clinical trials due to the slow rate of liver fibrosis progression. We aimed to develop a surrogate biomarker to predict future fibrosis progression."
!Series_summary	"Methods: A Fibrosis Progression Signature (FPS) was defined to predict fibrosis progression within 5 years in HCV and NAFLD patients with no to minimal fibrosis at baseline (n=421), and validated in an independent NAFLD cohort (n=78). The FPS was used to assess response to 13 candidate anti-fibrotics in organotypic ex vivo cultures of clinical fibrotic liver tissues (n=78), and cenicriviroc in NASH patients enrolled in a clinical trial (n=19, NCT02217475). A serum-protein-based surrogate FPS (FPSec) was developed and technically evaluated 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available.
is_gene_available = True  # Background info suggests gene expression profiling

# Define the keys for the variables in the sample characteristics dictionary
trait_row = None  # No indication that 'Liver_cirrhosis' is available in the sample characteristics
age_row = None  # No direct indication that 'age' is available
gender_row = None  # No direct indication that 'gender' is available

# Convert Trait function (if hypothetical values were found, this could be used)
def convert_trait(value):
    # Extract the actual value
    actual_value = value.split(':')[-1].strip()
    # Convert to a relevant binary value as an example (1 = Yes, 0 = No)
    if actual_value.lower() in ["yes", "1"]:
        return 1
    elif actual_value.lower() in ["no", "0"]:
        return 0
    else:
        return None

# Convert Age function (assuming age would be continuous if it were available)
def convert_age(value):
    # Extract the actual value
    try:
        actual_value = float(value.split(':')[-1].strip())
        return actual_value
    except ValueError:
        return None

# Convert Gender function
def convert_gender(value):
    # Extract the actual value
    actual_value = value.split(':')[-1].strip().lower()
    # Convert gender to binary (female = 0, male = 1)
    if actual_value == "male":
        return 1
    elif actual_value == "female":
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE182060', './preprocessed/Liver_cirrhosis/cohort_info.json', is_gene_available, trait_row is not None)

# As trait_row is None, the next substep for clinical feature extraction is skipped.


A new JSON file was created at: ./preprocessed/Liver_cirrhosis/cohort_info.json
